# Cálculo do TF-IDF para os termos dos textos descritivos

## Tokenização dos textos

In [1]:
# Imports
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 

import pandas as pd

#import spacy
import string

In [2]:
## Extraindo frequencias de termos dos textos descritivos do dataset.

In [3]:
df = pd.read_json('data_files/merged_datasets.json', orient='split')

In [4]:
corpus = df['Text']

In [5]:
# Apagando a variável df porque não será mais utilizada.
del df

In [6]:
# Quebrando cada texto em uma lista de sentenças.
corpus_sent_tokenized = [sent_tokenize(texto) for texto in corpus]

In [7]:
# Apagando a variável corpus porque não será mais utilizada.
del corpus

In [8]:
# Quebrando as sentenças em palavras, de modo que cada sentença seja transformada em lista de palavras.
corpus_word_tokenized = [[word_tokenize(sent) for sent in text] for text in corpus_sent_tokenized]

In [9]:
# Apagando a variável corpus_sent_tokenized porque não será mais utilizada
del corpus_sent_tokenized

## Remoção das stopwords e pontuações

In [10]:
english_stops = set(stopwords.words('english'))

In [11]:
def remove_stopwords_e_pontuacao(lista_words):
    tokens_limpos = [word.lower() for word in lista_words if (word.lower() not in english_stops and word not in string.punctuation)]
    return tokens_limpos

In [12]:
corpus_limpo = [[remove_stopwords_e_pontuacao(sent) for sent in text] for text in corpus_word_tokenized]

In [13]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_word_tokenized

In [27]:
corpus_limpo[1]

[['abstract',
  'background',
  'non-small',
  'cell',
  'lung',
  'cancer',
  'nsclc',
  'heterogeneous',
  'group',
  'disorders',
  'number',
  'genetic',
  'proteomic',
  'alterations'],
 ['c-cbl',
  'e3',
  'ubiquitin',
  'ligase',
  'adaptor',
  'molecule',
  'important',
  'normal',
  'homeostasis',
  'cancer'],
 ['determined',
  'genetic',
  'variations',
  'c-cbl',
  'relationship',
  'receptor',
  'tyrosine',
  'kinases',
  'egfr',
  'met',
  'functionality',
  'nsclc'],
 ['methods',
  'findings',
  'using',
  'archival',
  'formalin-fixed',
  'paraffin',
  'embedded',
  'ffpe',
  'extracted',
  'genomic',
  'dna',
  'show',
  'c-cbl',
  'mutations',
  'occur',
  'somatic',
  'fashion',
  'lung',
  'cancers'],
 ['c-cbl',
  'mutations',
  'mutually',
  'exclusive',
  'met',
  'egfr',
  'mutations',
  'however',
  'independent',
  'p53',
  'kras',
  'mutations'],
 ['normal/tumor',
  'pairwise',
  'analysis',
  'significant',
  'loss',
  'heterozygosity',
  'loh',
  'c-cbl',
  '

In [37]:
# Faz-se necessário juntar todas as palavras de cada texto em uma lista única, para que seja feita a contagem
def abre_listas(lista_de_listas):
    lista_dummy = []
    for lista in lista_de_listas:
        for elemento in lista:
            lista_dummy.append(elemento)
    return lista_dummy

In [38]:
doc = abre_listas(corpus_limpo[0])

In [39]:
doc

['cyclin-dependent',
 'kinases',
 'cdks',
 'regulate',
 'variety',
 'fundamental',
 'cellular',
 'processes',
 'cdk10',
 'stands',
 'one',
 'last',
 'orphan',
 'cdks',
 'activating',
 'cyclin',
 'identified',
 'kinase',
 'activity',
 'revealed',
 'previous',
 'work',
 'shown',
 'cdk10',
 'silencing',
 'increases',
 'ets2',
 'v-ets',
 'erythroblastosis',
 'virus',
 'e26',
 'oncogene',
 'homolog',
 '2',
 '-driven',
 'activation',
 'mapk',
 'pathway',
 'confers',
 'tamoxifen',
 'resistance',
 'breast',
 'cancer',
 'cells',
 'precise',
 'mechanisms',
 'cdk10',
 'modulates',
 'ets2',
 'activity',
 'generally',
 'functions',
 'cdk10',
 'remain',
 'elusive',
 'demonstrate',
 'cdk10',
 'cyclin-dependent',
 'kinase',
 'identifying',
 'cyclin',
 'activating',
 'cyclin',
 'cyclin',
 'orphan',
 'cyclin',
 'product',
 'fam58a',
 'whose',
 'mutations',
 'cause',
 'star',
 'syndrome',
 'human',
 'developmental',
 'anomaly',
 'whose',
 'features',
 'include',
 'toe',
 'syndactyly',
 'telecanthus',
 'a

In [40]:
documentos = [abre_listas(doc) for doc in corpus_limpo]

In [44]:
# Apagando a variável corpus_word_tokenized porque não será mais utilizada.
del corpus_limpo

## Stemming e Lemmatization

### Stemming

In [56]:
from nltk.stem import PorterStemmer
from nltk.stem import RegexpStemmer

In [49]:
porter_stemmer = PorterStemmer()

In [50]:
porter_stemmer.stem('cooking')

'cook'

In [51]:
porter_stemmer.stem('cookery')

'cookeri'

In [57]:
regexp_stemmer = RegexpStemmer('ing')

In [60]:
porter_stemmer.stem(regexp_stemmer.stem('cooking'))

'cook'

In [85]:
porter_stemmer.stem(regexp_stemmer.stem('activation'))

'activ'

In [86]:
porter_stemmer.stem(regexp_stemmer.stem('activating'))

'activat'

In [62]:
# Aplicando stemming, isto é, retirada de sufixos, para cada palavra na lista de documentos
docs_stemmed = [[porter_stemmer.stem(regexp_stemmer.stem(word)) for word in doc] for doc in documentos]

In [63]:
len(docs_stemmed)

3321

In [64]:
docs_stemmed[0]

['cyclin-depend',
 'kinas',
 'cdk',
 'regul',
 'varieti',
 'fundament',
 'cellular',
 'process',
 'cdk10',
 'stand',
 'one',
 'last',
 'orphan',
 'cdk',
 'activat',
 'cyclin',
 'identifi',
 'kinas',
 'activ',
 'reveal',
 'previou',
 'work',
 'shown',
 'cdk10',
 'silenc',
 'increas',
 'ets2',
 'v-et',
 'erythroblastosi',
 'viru',
 'e26',
 'oncogen',
 'homolog',
 '2',
 '-driven',
 'activ',
 'mapk',
 'pathway',
 'confer',
 'tamoxifen',
 'resist',
 'breast',
 'cancer',
 'cell',
 'precis',
 'mechan',
 'cdk10',
 'modul',
 'ets2',
 'activ',
 'gener',
 'function',
 'cdk10',
 'remain',
 'elus',
 'demonstr',
 'cdk10',
 'cyclin-depend',
 'kinas',
 'identifi',
 'cyclin',
 'activat',
 'cyclin',
 'cyclin',
 'orphan',
 'cyclin',
 'product',
 'fam58a',
 'whose',
 'mutat',
 'caus',
 'star',
 'syndrom',
 'human',
 'development',
 'anomali',
 'whose',
 'featur',
 'includ',
 'toe',
 'syndactyli',
 'telecanthu',
 'anogenit',
 'renal',
 'malform',
 'show',
 'star',
 'syndrome-associ',
 'cyclin',
 'mutant',


### Lemmatization

In [70]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/rafael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
wordnet_lemmatizer = WordNetLemmatizer()

In [72]:
print(wordnet_lemmatizer.lemmatize('mice'))
print(wordnet_lemmatizer.lemmatize('cacti'))  # plural da palavra cactus - cactuses (inglês) ou cacti (latin)
print(wordnet_lemmatizer.lemmatize('horses'))
print(wordnet_lemmatizer.lemmatize('wolves'))

mouse
cactus
horse
wolf


In [75]:
# Aplicando lemmatization, isto é, a aglutinação das palavras com o mesmo radical comum.
docs_stemmed_lemmatized = [[wordnet_lemmatizer.lemmatize(word) for word in doc] for doc in documentos]

In [91]:
docs_stemmed_lemmatized[0]

['cyclin-dependent',
 'kinase',
 'cdks',
 'regulate',
 'variety',
 'fundamental',
 'cellular',
 'process',
 'cdk10',
 'stand',
 'one',
 'last',
 'orphan',
 'cdks',
 'activating',
 'cyclin',
 'identified',
 'kinase',
 'activity',
 'revealed',
 'previous',
 'work',
 'shown',
 'cdk10',
 'silencing',
 'increase',
 'ets2',
 'v-ets',
 'erythroblastosis',
 'virus',
 'e26',
 'oncogene',
 'homolog',
 '2',
 '-driven',
 'activation',
 'mapk',
 'pathway',
 'confers',
 'tamoxifen',
 'resistance',
 'breast',
 'cancer',
 'cell',
 'precise',
 'mechanism',
 'cdk10',
 'modulates',
 'ets2',
 'activity',
 'generally',
 'function',
 'cdk10',
 'remain',
 'elusive',
 'demonstrate',
 'cdk10',
 'cyclin-dependent',
 'kinase',
 'identifying',
 'cyclin',
 'activating',
 'cyclin',
 'cyclin',
 'orphan',
 'cyclin',
 'product',
 'fam58a',
 'whose',
 'mutation',
 'cause',
 'star',
 'syndrome',
 'human',
 'developmental',
 'anomaly',
 'whose',
 'feature',
 'include',
 'toe',
 'syndactyly',
 'telecanthus',
 'anogenital'

## Cálculo do TF-IDF

In [20]:
# Função para calcular Term-frequency (frequencia do termo)
def TermFreq(documento, palavra):
    doc_length = len(documento)
    ocorrencias = len([w for w in documento if w == palavra])
    return ocorrencias / doc_length

In [79]:
TermFreq(docs_stemmed_lemmatized[0], 'cyclin')

0.02197530864197531

In [80]:
TermFreq(docs_stemmed_lemmatized[0], 'cancer')

0.002962962962962963

In [101]:
palavras_corpus = abre_listas(docs_stemmed_lemmatized)

In [102]:
# Criação de uma lista única contendo todas as palavras dos documentos para contagem dos termos no corpus
set_palavras_corpus = set(palavras_corpus)

In [103]:
len(set_palavras_corpus)

276083

In [105]:
len(palavras_corpus)

21194984

### Estabelecendo um contador para apurar a frequência das palavras

In [115]:
from collections import Counter

In [116]:
%%time
contador = Counter(palavras_corpus)

CPU times: user 2.64 s, sys: 16 ms, total: 2.65 s
Wall time: 2.65 s


In [117]:
len(contador)

276083

In [118]:
contador

Counter({'cyclin-dependent': 719,
         'kinase': 70231,
         'cdks': 576,
         'regulate': 3285,
         'variety': 3159,
         'fundamental': 611,
         'cellular': 12078,
         'process': 6717,
         'cdk10': 78,
         'stand': 179,
         'one': 39369,
         'last': 2400,
         'orphan': 74,
         'activating': 13566,
         'cyclin': 11646,
         'identified': 38033,
         'activity': 83248,
         'revealed': 12185,
         'previous': 11240,
         'work': 3841,
         'shown': 43856,
         'silencing': 1459,
         'increase': 14546,
         'ets2': 232,
         'v-ets': 4,
         'erythroblastosis': 33,
         'virus': 1981,
         'e26': 19,
         'oncogene': 6932,
         'homolog': 1259,
         '2': 79657,
         '-driven': 25,
         'activation': 49009,
         'mapk': 6133,
         'pathway': 38911,
         'confers': 1407,
         'tamoxifen': 316,
         'resistance': 27444,
         'bre

In [119]:
contador['revealed']

12185

In [ ]:
# Função para calcular a Frequência Inversa de Documentos
def InverseDocumentFrequency(word):
    N = len(palavras_corpus)
    try:
        df = contador[word] + 1
    except:
        df = 1
    return np.log(N/df)